# Анализ кампании по сбору подписей за Бориса Надеждина

В связи с тем, что только команда Б.Б. Надеждина выкладывала в открытый доступ данные о количестве собранных подписей по каждому региону страны отдельно, получилось собрать и обработать полученные результаты.

## Импорт необходимых библиотек

In [1]:
# Блок 0: Скрыть уведомления
import warnings
warnings.filterwarnings('ignore')

# Блок 1: Анализ данных и визуализация
import pandas as pd
import re

# Блок 2: Веб-скрапинг
from bs4 import BeautifulSoup
import requests

# Блок 3: Отображение HTML-контента
from IPython.display import IFrame

# Блок 4: Работа с геопространственными данными
import geopandas as gpd

# Блок 5: Создание карт с помощью Folium
import folium
from branca.colormap import linear

## Сбор данных

Для получения информации о том, в каких регионах страны были собраны подписи и в каком количестве, необходимо выгрузить данные с сайта кампании Б.Б.Надеждина и привести их в необходимые для дальнейшей работы вид. 

In [2]:
def get_html(url):
    # забираем html-dom
    result = requests.get(url)

    return result.text

def get_block(html):

    soup = BeautifulSoup(html, 'lxml')
    # разбираем на блоки адресс-подпись для удобной комбинации
    blocks = soup.findAll('div', class_='addresses-page__region')

    adresses = []
    signatures = []

    for el in blocks:
        # в каждом блоке точно есть регион, забираем его
        adress = el.find('div', class_='card region-card region-card--addresses-page').find('h3', class_='subheading').text
        adresses.append(adress)
        # в блоке еще может не быть подписей
        try:
            # имеем только один спан в диве с таким классом 
            sign = el.find('div', class_='progressbar addresses-page__progressbar').find('span').text
            sign = re.findall(r'\d+', sign)[0] # забираем только число собранных подписей
            signatures.append(sign)
        except:
            # placeholder
            signatures.append('nan')


    return adresses, signatures


def get():

    html = get_html('https://nadezhdin2024.ru/addresses') # Адрес сайта, используемый 26/01/24 для мониторинга собранных подписей
    # html = get_html('https://nadezhdin2024.ru/signatures') # Адрес сайта, используемый после 30/01/24 для мониторинга обработанных подписей
    info = get_block(html)
    return info

adresses, signs = get()
signature = pd.DataFrame({'Регион': adresses, 'Подписи': signs})
signature.to_csv('signature_new.csv', index=False)

## Загрузка данных
В связи с тем, что штаб закрыл доступ к актуальным данным после решения ЦИКа от 08/02/24 от закрытии кампании кандидата, в дальнейшем исследовании будет использоваться архивная информация от 27/01/24. 

Важно отметить, что в связи с использованием архивных данных, в таких регионах как Москва будет наблюдаться существенный "перевес" голосов, т.к. отмечены только штабы, в которых собраны данные, а не регионы, к которым относятся люди, поставившие подписи. 

In [3]:
signature = pd.read_csv('old_signature.csv')  # Данные о собранных подписях

In [4]:
# Загрузка данных о регионах России
reg_inf = pd.read_csv('geo_1.csv')  # Правильные сопоставления регионов и номеров 
rus_info_df = pd.read_csv('regions-info.csv')  # Вся остальная географическая информация
# Загрузка географических данных России
rus_bnd_gdf = gpd.read_file('geo.json')

## Предобработка данных

### Подготовка геоданных

In [5]:
rus_info_df = rus_info_df[rus_info_df['Region'] != 'Россия']

In [6]:
# Combining with regions info
right = rus_info_df[['Region_ID', 'Region', 'Latitude', 'Longitude']]

# Присоединение right к reg_inf по столбцу region_name и Region
geo_df = reg_inf.merge(right, left_on='region_name', right_on='Region', how='left')

In [7]:
missing_regions = right[~right['Latitude'].isin(geo_df['Latitude'])]
missing_regions

,Region_ID,Region,Latitude,Longitude
7,2,Республика Башкортостан,54.474755,55.978458
82,79,Еврейская автономная область,48.560161,132.277566
84,87,Чукотский автономный округ,66.000647,169.490087
85,83,Ненецкий автономный округ,67.678325,57.062685


In [8]:
# Manual fixes
geo_df.loc[geo_df.Region == 'Москва', ['Latitude', 'Longitude']] = (55.7522, 37.6220)
geo_df.loc[geo_df.Region == 'Санкт-Петербург', ['Latitude', 'Longitude']] = (59.8917, 30.2673)

geo_df.loc[geo_df.region_name == 'Pеспублика Башкортостан', ['Latitude', 'Longitude', 'Region']] = (
    54.474755, 55.978458, 'Республика Башкортостан')
geo_df.loc[geo_df.region_name == 'Республика Чувашия', ['Latitude', 'Longitude', 'Region']] = (
    56.142431, 47.254454, 'Республика Чувашия')
geo_df.loc[geo_df.region_name == 'Еврейская АО', ['Latitude', 'Longitude', 'Region']] = (
    48.560161, 132.277566, 'Еврейская АО')
geo_df.loc[geo_df.region_name == 'Чукотский АО', ['Latitude', 'Longitude', 'Region']] = (
    66.000647, 169.490087, 'Чукотский АО')
geo_df.loc[geo_df.region_name == 'Ненецкий АО', ['Latitude', 'Longitude', 'Region']] = (
    67.678325, 57.062685, 'Ненецкий АО')


In [9]:
geo_df = geo_df[['id', 'region_name', 'eng_name', 'Latitude', 'Longitude']]
# Переименование столбцов
geo_df.rename(columns={'id': 'id', 'region_name': 'region', 'eng_name': 'region_eng', 'Latitude': 'latitude',
                       'Longitude': 'longitude'}, inplace=True)

### Подготовка данных о подписях
Для начала ноебходимо убрать из списка все подписи, которые были собраны за пределами РФ

In [10]:
# Оформление списка
to_del = ['Австрия', 'Аргентина', 'Армения', 'Бельгия', 'Болгария', 'Германия', 'Греция', 'Грузия', 'Дания', 'Израиль',
          'Индонезия', 'Ирландия', 'Испания', 'Италия', 'Казахстан', 'Кипр', 'Кыргызстан', 'Латвия', 'Литва', 'Молдова',
          'Нидерланды', 'Норвегия', 'ОАЭ', 'Португалия', 'США', 'Сербия', 'Соединённое Королевство', 'Тайланд',
          'Турция', 'Узбекистан', 'Финляндия', 'Франция', 'Черногория', 'Чехия', 'Швейцария', 'Швеция', 'Шри-Ланка',
          'Япония']

# Удаление регионов из DataFrame
signature = signature[~signature['Регион'].isin(to_del)]
signature.loc[:, 'Подписи'] = signature['Подписи'].fillna(0)

Добавим столбцы с аналитикой собранных подписей:
- сколько всего собрано подписей, с учетом ограничений по количеству на один регион;
- сколько региону не хватает до максимального значения;
- насколько регион собрал больше лимита.  

In [11]:
# Создаем столбец "собрано"
signature['собрано'] = signature['Подписи'].apply(lambda x: min(x, 2500))

# Создаем столбец "необходимо"
signature['необходимо'] = 2500 - signature['собрано']

# Создаем столбец "сверх"
signature['сверх'] = signature['Подписи'].apply(lambda x: max(x - 2500, 0))

In [12]:
# Переименование столбцов
signature.rename(columns={'Регион': 'region', 'Подписи': 'signatures', 'собрано': 'collected', 'необходимо': 'needed',
                          'сверх': 'extra'}, inplace=True)

Переименуем регионы в соответствии с официальными названиями.

In [13]:
replace_dict = {
    'Адыгея': 'Республика Адыгея',
    'Башкортостан': 'Республика Башкортостан',
    'Бурятия': 'Республика Бурятия',
    'Алтай': 'Республика Алтай',
    'Дагестан': 'Республика Дагестан',
    'Ингушетия': 'Республика Ингушетия',
    'Калмыкия': 'Республика Калмыкия',
    'Карелия': 'Республика Карелия',
    'Коми': 'Республика Коми',
    'Марий Эл': 'Республика Марий Эл',
    'Мордовия': 'Республика Мордовия',
    'Саха (Якутия)': 'Республика Саха (Якутия)',
    'Татарстан': 'Республика Татарстан',
    'Тыва': 'Республика Тыва',
    'Удмуртия': 'Удмуртская Республика',
    'Хакасия': 'Республика Хакасия',
    'Чувашская Республика': 'Республика Чувашия',
    'Ханты-Мансийский автономный округ': 'Ханты-Мансийский АО'
}

signature['region'] = signature['region'].replace(replace_dict)

Добавим информацию о регионах к подписям

In [14]:
# new_df = signature.merge(reg_inf[['id', 'region_name']], left_on='region_name', right_on='region', how='left')
signature = pd.merge(signature, geo_df, left_on='region', right_on='region', how='right')

Заполним пропуски и удалим столбцы-дубликаты

In [15]:
signature.loc[:, 'signatures'] = signature['signatures'].fillna(0)

signature.loc[signature['signatures'] == 0, ['collected', 'extra']] = 0
signature.loc[signature['signatures'] == 0, 'needed'] = 2500

Отсортируем столбцы в удобном порядке

In [16]:
signature = signature[
    ['id', 'region', 'region_eng', 'collected', 'needed', 'signatures', 'extra', 'latitude', 'longitude']]

## Построение графиков

Построим тепловую карту по собранным подписям, с учетом ограничений на один регион

In [17]:
def embed_map(m, file_name):
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [18]:
# Regions mapping
rus_gdf = gpd.GeoDataFrame(signature, geometry=gpd.points_from_xy(geo_df.longitude, geo_df.latitude))
rus_gdf.crs = {'init': 'epsg:4326'}

rus_shape = rus_bnd_gdf[['NAME_1', 'TYPE_1', 'ID_1', 'geometry']]
rus_gdf = gpd.sjoin(rus_gdf, rus_shape, how="left", op='within')

In [19]:
sing_map = folium.Map(
    location=[64.0914, 101.6016],
    zoom_start=3
)

colormap = linear.RdYlGn_09.scale(rus_gdf.collected.min(), rus_gdf.collected.max())


def color_mapper(id):
    row = rus_gdf[rus_gdf.ID_1 == id].reset_index()
    return row.collected.iloc[0]


folium.GeoJson(
    rus_bnd_gdf,
    name='rusjson',
    style_function=lambda feature: {
        'fillColor': colormap(color_mapper(feature['properties']['ID_1'])),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(sing_map)

for i in range(len(rus_gdf)):
    folium.Circle(
        radius=20000,
        location=[rus_gdf.latitude[i], rus_gdf.longitude[i]],
        popup=rus_gdf.region[i] + '\nСобрали: {}, \nСверх: {}, \nНеобходимо: {}'.format(rus_gdf.collected[i],
                                                                                        rus_gdf.extra[i],
                                                                                        rus_gdf.needed[i]),
        color='crimson',
        fill=True,
    ).add_to(sing_map)

embed_map(sing_map, 'sing_map.html')

Полученную карту можно посмотреть по ссылке [https://andaisrin.github.io/kaggle_competitions/sing_map.html](https://andaisrin.github.io/kaggle_competitions/sing_map.html)